In [1]:
# to highlight part of a pdf file, use the following command:
# !python -m pip install --upgrade pymupdf
import fitz


In [2]:
from functions import extract_all_text2
import embedding_functions

/home/gsilvi/miniforge3/envs/PaperOracle/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [3]:
all_tex = extract_all_text2('papers/2101.00689/Npi_paper.tex')

In [4]:
final_text = embedding_functions.texStripper2(all_tex,'title','abstract') # refine the text

KEYS: dict_keys(['author', 'email', 'affiliation', 'title', 'date', 'abstract', 'table', 'table*', 'equation', 'align', 'enumerate', 'figure', 'figure*', 'eqnarray', 'gather', 'footnotesize'])
SECTIONS: dict_keys(['-', 'Introduction', 'Gauge Ensemble', 'Interpolating Operators', 'Wick contractions', 'Spectra results', 'L{\\"u}scher quantization conditions}\n', 'Results for the scattering amplitudes', 'Parametrizations used', 'Fit procedure and results', 'Conclusions', 'Acknowledgments', 'One-to-one mapping of energy levels to phase shifts in irreps without mixing between $J=1/2$ and $J=3/2$', 'Transformation properties of operators', '\\texorpdfstring{Matrices $\\mathcalM'])
author [('Giorgio Silvi', 58), ('\\mbox\\affiliation{\\MAINZ', 63), ('Constantia Alexandrou', 66), ('Stefan Krieg', 71), ('Luka Leskovec', 75), ('Stefan Meinel', 79), ('John Negele', 82), ('Marcus Petschlies', 85), ('Andrew Pochinsky', 88), ('Gumaro Rendon', 91), ('Sergey Syritsyn', 95), ('Antonino Todaro', 99)]
em

Token indices sequence length is longer than the specified maximum sequence length for this model (2093 > 1024). Running this sequence through the model will result in indexing errors


Phrase too long: 2889  trying to split it into 1 phrases
Phrase too long: 6334  trying to split it into 1 phrases
Phrase too long: 6031  trying to split it into 1 phrases
Phrase too long: 4059  trying to split it into 1 phrases
Phrase too long: 8735  trying to split it into 1 phrases


In [5]:
import pandas as pd
for file in final_text.keys():
    df = pd.DataFrame(final_text[file]['full'], columns=['Phrase','Line', 'LineEnd'])

# fix case where you have \begin{something} and \end{something} on different lines
# reuse line.startswith("\\begin{") && extract_begin_to_end(complete_text2,l,'\\end{'+keyword)
# .\n is good for most cases  unless the above happen.



In [8]:
#set the environment variable TOKENIZERS_PARALLELISM=true
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

# extract line 36 from df
import subprocess
line_in_df = 36

start = df.iloc[line_in_df].Line
print(df.iloc[line_in_df])

print(start)
result = ''
collection = list()
gen_dict = {}
for row in range(start,start+1):
    print(row)
    result += subprocess.run(['synctex', 'view', '-i',f'{row}:0:papers/2101.00689/Npi_paper.tex','-o','papers/2101.00689/Npi_paper.pdf' ], stdout=subprocess.PIPE).stdout.decode('utf-8')

for stuck in result.split('\n'):
    
    if ':' in stuck:
        key, value = stuck.split(':')
        gen_dict[key] = value
        if key=='after':
            collection.append(gen_dict)
            gen_dict = {}
            


for c in collection:    
    print(c)

# height from y to v
# width from x to h+W

Phrase    [[Introduction]]The $\Delta(1232)$ (in the fol...
Start                                                   123
End                                                     125
Name: 36, dtype: object
123 125
123
124
125
{'Output': 'papers/2101.00689/Npi_paper.pdf', 'Page': '1', 'x': '71.999985', 'y': '500.144440', 'h': '71.999985', 'v': '500.144440', 'W': '225.033615', 'H': '10.460772', 'before': '', 'offset': '-1', 'middle': '', 'after': ''}
{'Output': 'papers/2101.00689/Npi_paper.pdf', 'Page': '1', 'x': '132.299362', 'y': '500.144440', 'h': '71.999985', 'v': '500.144440', 'W': '225.033615', 'H': '10.460772', 'before': '', 'offset': '-1', 'middle': '', 'after': ''}
{'Output': 'papers/2101.00689/Npi_paper.pdf', 'Page': '1', 'x': '132.299362', 'y': '500.144440', 'h': '71.999985', 'v': '500.144440', 'W': '225.033615', 'H': '10.460772', 'before': '', 'offset': '-1', 'middle': '', 'after': ''}
{'Output': 'papers/2101.00689/Npi_paper.pdf', 'Page': '1', 'x': '132.299362', 'y': '489.68365

In [9]:
### READ IN PDF
doc = fitz.open("papers/2101.00689/Npi_paper.pdf")
# text_instances = [fitz.Rect(132,489, 296,500)]
text_instances = set()
text_instances2 = set()

for stuck in collection:
        
    if float(stuck['y'])<float(stuck['v']):
        text_instances2.add((stuck['Page'],
                              float(stuck['x']),
                              float(stuck['y']),
                              float(stuck['h'])+float(stuck['W']),
                              float(stuck['v']))
                              )
    
    text_instances.add((stuck['Page'],
                        float(stuck['h']),
                        float(stuck['v'])-float(stuck['H']),
                        float(stuck['h'])+float(stuck['W']),
                        float(stuck['v']))
                        )
    ### HIGHLIGHT
for page in doc:
    ### SEARCH
    print(page.number)
    
    for inst in text_instances:
        # divide the tuple inst into its components
        (pageNum, x0, y0, x1, y1) = inst 
        if pageNum==str(page.number+1):
            print(inst)
            highlight = page.add_highlight_annot(fitz.Rect(x0, y0, x1, y1))
            highlight.update()
    for inst in text_instances2:
        # divide the tuple inst into its components
        (pageNum, x0, y0, x1, y1) = inst 
        if pageNum==str(page.number+1):
            print(inst)
            highlight = page.add_underline_annot(fitz.Rect(x0, y0, x1, y1))
            highlight.update()
### OUTPUT

doc.save("output.pdf", garbage=4, deflate=True, clean=True)

0
('1', 71.999985, 518.30268, 297.0336, 528.26532)
('1', 71.999985, 597.450341, 297.0336, 611.156921)
('1', 71.999985, 541.770239, 297.0336, 550.625916)
('1', 71.999985, 564.684301, 297.0336, 573.539978)
('1', 71.999985, 552.673774, 297.0336, 562.636414)
('1', 71.999985, 587.598425, 297.0336, 596.454102)
('1', 71.999985, 623.716955, 297.0336, 632.572632)
('1', 71.999985, 576.141393, 297.0336, 584.99707)
('1', 71.999985, 489.68366799999995, 297.0336, 500.14444)
('1', 71.999985, 612.259924, 297.0336, 621.115601)
('1', 71.999985, 634.6205510000001, 297.0336, 644.583191)
('1', 71.999985, 530.313207, 297.0336, 539.168884)
('1', 94.692703, 548.688721, 297.0336, 550.625916)
('1', 209.293549, 607.721375, 297.0336, 611.156921)
('1', 135.931763, 632.129883, 297.0336, 632.572632)
('1', 210.489059, 603.798767, 297.0336, 611.156921)
('1', 98.013626, 630.635437, 297.0336, 632.572632)
('1', 93.862488, 607.721375, 297.0336, 611.156921)
('1', 189.36734, 604.106018, 297.0336, 611.156921)
('1', 104.93210